In [ ]:
#Imports
import pandas as pd
import numpy as np
from goatools.base import get_godag
from goatools.anno.gaf_reader import GafReader
from goatools.gosubdag.gosubdag import GoSubDag
from collections import defaultdict
import pickle
#Get the GO terms
godag = get_godag("go-basic.obo", optional_attrs='relationship')
#Get the GO Annotation File
gogaf = GafReader("goa_yeast.gaf")
#Build pairs and sl columns in gene2sl
genes     = pd.read_csv('gsets_1', header=None, names=["genes"])
sl_lables = pd.read_csv('sl_1', header=None, names=["sl"])
gene2sl   = pd.concat([genes, sl_lables], axis=1)
#gene2sl   = gene2sl.set_index('genes')
#Now get a gene to GO id dict.
gene2goid = defaultdict(set)
for assoc in gogaf.associations:
    for syn in assoc.DB_Synonym:
        gene2goid[syn].add(assoc.GO_ID)

In [ ]:
%%time
import numpy as np
from scipy.stats.stats import pearsonr
np.random.seed(0)  # seed for reproducibility

x = np.random.randint(10, size=6000)  # One-dimensional array
y = np.random.randint(10, size=6000)  # One-dimensional array
print(pearsonr(x,y))

In [1]:
%%time
import pandas as pd
# mmc = pd.read_excel("mmc5.xlsx")
# mmc.head(25)
tcga = pd.read_csv("tcga_data.csv", sep="\t")
tcga.head(25)

FileNotFoundError: [Errno 2] File b'tcga_data.csv' does not exist: b'tcga_data.csv'

In [ ]:
tcga.shape

In [ ]:
horlbeck_data = pd.read_excel("mmc5.xlsx", sheet_name=1, header=None, skiprows=4, usecols=[0,1,6])
horlbeck_data.set_index([0,1]).sort_index().dropna()


In [ ]:
horlbeck_data[6].describe()

In [ ]:
horlbeck_data.shape

In [ ]:
dir(gogaf)

In [ ]:
dir(godag)

In [ ]:
term = godag.get('GO:0010468')
term

In [ ]:
p1 = term.parents
p1

In [ ]:
%%time
#%%prun

#Now build the hierarchy, meaning each node is represented by the no. of all its descendants
genes2terms = pd.DataFrame()
gene2terms = {}
list_genes2terms = []
for index, row in gene2sl.iterrows():
    for gene in row['genes'].split("$"):
        #No such gene, build it
        if gene not in gene2terms:
            gene2terms[gene] = {}
            genebiogoid = []
            go_ids = gene2goid.get(gene)
            if go_ids is None:
                print("Could not find the gene {} in GAF".format(gene))               
            else:
                for go_id in go_ids:
                    term = godag.get(go_id)
                    if term.namespace == 'biological_process':                     
                        #Get the leaf GO term
                        genebiogoid.append(go_id)  
                        #Mark the visitation of that leaf
                        if go_id not in gene2terms[gene]:
                            gene2terms[gene][go_id] = 1
                        else:
                            gene2terms[gene][go_id] += 1
            # Create a subset of the GO DAG which contains:
            # The common GO terms that are 'is_a' for the gene, by setting relationships=False
            gosubdag = GoSubDag(genebiogoid, godag, relationships=False, prt=False) 
            ancestors = defaultdict(set)
            if go_id in gosubdag.rcntobj.go2parents:
                for p_go in gosubdag.rcntobj.go2parents[go_id]:
                    dict_anc = gosubdag.go2nt[p_go]._asdict()
                    ancestors[dict_anc['depth']].add(dict_anc['GO'])                            
            #Sum up all the ancestors values (the root is marked as depth 0)
            count_genes = 1  
            for level in range(len(ancestors) - 1, -1, -1):
                for GO in ancestors[level]:
                    if GO not in gene2terms[gene]:
                        gene2terms[gene][GO] = count_genes
                    else:
                        gene2terms[gene][GO] += count_genes                                
                count_genes += len(ancestors[level])                          
    #Build the combination of the 2 genes
    g1, g2 = row['genes'].split("$")
    gene1 = gene2terms[g1]
    gene2 = gene2terms[g2]         
    genesdict = {key: gene1.get(key, 0) + gene2.get(key, 0)
                 for key in set(gene1) | set(gene2)}
    genesdict['genes'] = row['genes']
    genesdict['sl'] = row['sl']
    list_genes2terms.append(genesdict)    
    if index == 100:
        break
#Add the final data and clean NaN
genes2terms = genes2terms.append(list_genes2terms, ignore_index=True)
genes2terms = genes2terms.fillna(0)

In [ ]:
with open("test.txt", "wb") as fp:   #Pickling
    pickle.dump(list_genes2terms, fp) 

In [ ]:
with open("list_genes2terms.txt", "rb") as fp:   # Unpickling
     b = pickle.load(fp)

In [ ]:
len(b)

In [ ]:
genes2terms

In [ ]:
v2_g2t

In [ ]:
genes2terms = pd.DataFrame()
gene2terms = {}
for index, row in gene2sl.iterrows():
    genebiogoid = []
    for gene in row['genes'].split("$"):
        #No such gene, build it
        if gene not in gene2terms:
            go_ids = gene2goid.get(gene)
            if go_ids is None:
                print("Could not find the gene {} in GAF".format(gene))               
            else:
                for go_id in go_ids:
                    term = godag.get(go_id)
                    if term.namespace == 'biological_process':
                        #Get the leaf GO term
                        genebiogoid.append(go_id)                        
            #Build the combination of the 2 genes
            # Create a subset of the GO DAG which contains:
            # The selected GO term and the 'is_a' GO terms above it, by setting relationships=False
            gene2terms[gene] = {}
            gosubdag = GoSubDag(genebiogoid, godag, relationships=False, prt=False) 
            ancestors = defaultdict(set)
            for p_genegoid in genebiogoid:
                count_genes = 1 
                if p_genegoid not in genesdict:
                    gene2terms[p_genegoid] = count_genes
                else:
                    gene2terms[p_genegoid] += count_genes 
                if p_genegoid in gosubdag.rcntobj.go2parents:
                    for p_go in gosubdag.rcntobj.go2parents[p_genegoid]:
                            dict_anc = gosubdag.go2nt[p_go]._asdict()
                            ancestors[dict_anc['depth']].add(dict_anc['GO'])
                #Sum up all the ancestors values
            for level in range(len(ancestors) - 1, -1, -1):
                for GO in ancestors[level]:
                    if GO not in genesdict:
                        gene2terms[GO] = count_genes
                    else:
                        gene2terms[GO] += count_genes                                
                    count_genes += len(ancestors[level])  
    g1, g2 = row['genes'].split("$")
    gene1 = gene2terms[g1]
    gene2 = gene2terms[g2]         
    genesdict = {key: gene1.get(key, 0) + gene2.get(key, 0)
                 for key in set(gene1) | set(gene2)}
    genesdict['genes'] = row['genes']
    genesdict['sl'] = row['sl']
    #Add the final line and clean NaN
    genes2terms = genes2terms.append(genesdict, ignore_index=True)
    genes2terms = genes2terms.fillna(0)
    break
    print(index)
    if index == 100: break

In [ ]:
%matplotlib notebook
import numpy as np
from scipy import interp
import matplotlib.pyplot as plt
from sklearn import svm, datasets
from sklearn.metrics import auc
from sklearn.metrics import plot_roc_curve
from sklearn.model_selection import StratifiedKFold

# #############################################################################
# Data IO and generation

# Import some data to play with
iris = datasets.load_iris()
X = iris.data
y = iris.target
X, y = X[y != 2], y[y != 2]
n_samples, n_features = X.shape

# Add noisy features
random_state = np.random.RandomState(0)
X = np.c_[X, random_state.randn(n_samples, 200 * n_features)]

# #############################################################################
# Classification and ROC analysis

# Run classifier with cross-validation and plot ROC curves
cv = StratifiedKFold(n_splits=6)
classifier = svm.SVC(kernel='linear', probability=True,
                     random_state=random_state)

tprs = []
aucs = []
mean_fpr = np.linspace(0, 1, 100)

fig, ax = plt.subplots()
for i, (train, test) in enumerate(cv.split(X, y)):
    classifier.fit(X[train], y[train])
    viz = plot_roc_curve(classifier, X[test], y[test],
                         name='ROC fold {}'.format(i),
                         alpha=0.3, lw=1, ax=ax)
    interp_tpr = interp(mean_fpr, viz.fpr, viz.tpr)
    interp_tpr[0] = 0.0
    tprs.append(interp_tpr)
    aucs.append(viz.roc_auc)

ax.plot([0, 1], [0, 1], linestyle='--', lw=2, color='blue',
        label='Chance', alpha=.8)

mean_tpr = np.mean(tprs, axis=0)
mean_tpr[-1] = 1.0
mean_auc = auc(mean_fpr, mean_tpr)
std_auc = np.std(aucs)
ax.plot(mean_fpr, mean_tpr, color='green',
        label=r'Mean ROC (AUC = %0.2f $\pm$ %0.2f)' % (mean_auc, std_auc),
        lw=2, alpha=.8)

std_tpr = np.std(tprs, axis=0)
tprs_upper = np.minimum(mean_tpr + std_tpr, 1)
tprs_lower = np.maximum(mean_tpr - std_tpr, 0)
ax.fill_between(mean_fpr, tprs_lower, tprs_upper, color='grey', alpha=.2,
                label=r'$\pm$ 1 std. dev.')

ax.set(xlim=[-0.05, 1.05], ylim=[-0.05, 1.05],
       title="Receiver operating characteristic for {}".format(classifier.__class__.__name__))
ax.legend(loc="lower right")
plt.show()

In [ ]:
import sklearn

print('The scikit-learn version is {}.'.format(sklearn.__version__))

In [ ]:
#pip install -U scikit-learn

In [ ]:
from goatools.godag.go_tasks import get_go2parents

GO_ID = 'GO:0008150'  # regulation of metabolic process - 0019222
optional_relationships = set()
go2parents_isa = get_go2parents(godag, optional_relationships)
print(go2parents_isa)

In [ ]:
def ppp():
    print(godag['GO:0008150'])

In [ ]:
tmp_gosubdag = GoSubDag('GO:0048522', godag, relationships=False, prt=False)
tmp_gosubdag.rcntobj.go2parents['GO:0048522']

In [ ]:
def build_godag_isa():
        #Now build the hierarchy, meaning each node is represented by the no. of all its descendants
    genes2terms = pd.DataFrame()
    gene2terms = {}
    for index, row in gene2sl.iterrows():
        for gene in row['genes'].split("$"):
            #No such gene, build it
            if gene not in gene2terms:
                go_ids = gene2goid.get(gene)
                if go_ids is None:
                    print("Could not find the gene {} in GAF".format(gene))               
                else:
                    gene2terms[gene] = {}
                    for go_id in go_ids:
                        #go_id = 'GO:0048522'
                        term = godag.get(go_id)
                        if term.namespace == 'biological_process':
                            # Create a subset of the GO DAG which contains:
                            # The selected GO term and the 'is_a' GO terms above it, by setting relationships=False
                            gosubdag = GoSubDag(go_id, godag, relationships=False, prt=False) 
                            ancestors = defaultdict(set)
                            if go_id in gosubdag.rcntobj.go2parents:
                                for p_go in gosubdag.rcntobj.go2parents[go_id]:
                                    dict_anc = gosubdag.go2nt[p_go]._asdict()
                                    ancestors[dict_anc['depth']].add(dict_anc['GO'])
                            #print(ancestors)       
                            #Get the leaf GO term
                            if go_id not in gene2terms[gene]:
                                gene2terms[gene][go_id] = 1
                            else:
                                gene2terms[gene][go_id] += 1
                            #Sum up all the ancestors values
                            count_genes = 1  
                            for level in range(len(ancestors) - 1, -1, -1):
                                for GO in ancestors[level]:
                                    if GO not in gene2terms[gene]:
                                        gene2terms[gene][GO] = count_genes
                                    else:
                                        gene2terms[gene][GO] += count_genes                                
                                count_genes += len(ancestors[level])  

        #Build the combination of the 2 genes
        g1, g2 = row['genes'].split("$")
        gene1 = gene2terms[g1]
        gene2 = gene2terms[g2]             
        genesdict = {key: gene1.get(key, 0) + gene2.get(key, 0)
                     for key in set(gene1) | set(gene2)}
        #print(genesdict)
        genesdict['genes'] = row['genes']
        genesdict['sl'] = row['sl']
        #Add the final line and clean NaN
        genes2terms = genes2terms.append(genesdict, ignore_index=True)
        genes2terms = genes2terms.fillna(0)
        #print(genes2terms)
        #break
        if index == 10: break

In [ ]:
%prun build_godag_isa()

In [ ]:
genes2terms

In [ ]:
import numpy as np
outfile = 'mnmc_lite.features.npz'
npzfile = np.load(outfile)

In [ ]:
npzfile.files

In [ ]:
npzfile['features'].shape

In [ ]:
npzfile['overlay_features'].shape

In [ ]:
df22  = pd.DataFrame()
ll = []
ll.append({'GO:001':1, 'GO:004':4})
ll.append({'GO:003':3, 'GO:005':5})
df22 = df22.append(ll, ignore_index=True)
df22 = df22.fillna(0)
df22

In [ ]:
# import pandas as pd
# import numpy as np

# df = pd.DataFrame([{'c1':10, 'c2':100}, {'c1':11,'c2':110}, {'c1':12,'c2':120}])
# for index, row in df.iterrows():
#     print(str(index) + '\n')
#     print(row['c1'], row['c2'])
# df = pd.DataFrame(columns=['lib', 'qty1', 'qty2', 'qty3', 'qty4'])
# df2 = pd.DataFrame(columns=['lib', 'qty1', 'qty2'])
# for i in range(5):
#     df2.loc[i] = ['name' + str(i)] + list(randint(10, size=2))
# df3 = pd.DataFrame(columns=['lib', 'qty1', 'qty2'])
# for i in range(5):
#     df3.loc[i] = ['name' + str(i)] + list(randint(10, size=2))
# df.loc[0] = df2[df2.lib == 'name0'] + df3[df3.lib == 'name0'] 
# df
# df2
# df3

In [ ]:
# List of Tuples
students = [ ('jack', 34, 'Sydeny' , 'Australia') ,
             ('Riti', 30, 'Delhi' , 'India' ) ,
             ('Vikas', 31, 'Mumbai' , 'India' ) ,
             ('Neelu', 32, 'Bangalore' , 'India' ) ,
             ('John', 16, 'New York' , 'US') ,
             ('Mike', 17, 'las vegas' , 'US')  ]
#print('{NS} {GO} D{depth:02} {GO_name}'.format(**gosubdag.go2nt[p_go]._asdict()))
#Create a DataFrame object
dfObj = pd.DataFrame(students, columns = ['Name' , 'Age', 'City' , 'Country'], index=['a', 'b', 'c' , 'd' , 'e' , 'f'])
# Pass the row elements as key value pairs to append() function 
dfObj = dfObj.append({'Name' : 'Sahil' , 'City' : 'my butt'} , ignore_index=True)

In [ ]:
ty = defaultdict(set)
ty['YW2'] = {'go0001':1, 'go0002':3}
'YW2' not in ty

In [ ]:
dict1 = {'a': 5, 'b': 7}
dict2 = {'a': 3, 'c': 1}

result = {key: dict1.get(key, 0) + dict2.get(key, 0)
          for key in set(dict1) | set(dict2)}
result['genes'] = 'YSYD$YSHS2'
result['sl'] = 0
ff = pd.DataFrame()
ff = ff.append(result, ignore_index=True)
ff = ff.append({}, ignore_index=True)
ff = ff.fillna(0)

In [ ]:
from sklearn.metrics import roc_curve
from sklearn.metrics import roc_auc_score
import numpy as np
import pandas as pd
from scipy import interp
import matplotlib.pyplot as plt
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import auc
from sklearn.metrics import plot_roc_curve
from sklearn.model_selection import StratifiedKFold

genes2terms = pd.DataFrame()
genes2terms = pd.read_pickle("genes2terms.pkl")
genes2terms = genes2terms.set_index('genes')
#Get the standard model to which we are comparing
standard_model = pd.read_csv('all.onto_genes.features.cv_all.rf.predictions', header=None, names=["prob"])
standard_model['sl'] = -1
standard_model.loc[standard_model['prob'] >= 0.5,'sl'] = 1
standard_model.loc[standard_model['prob'] < 0.5,'sl'] = 0
standard_model['pairs'] = genes2terms.index.values
standard_model = standard_model.set_index('pairs')
standard_model = standard_model.head(5000)
# Set the X, y data
X = genes2terms.drop(['sl'], axis=1)
y = genes2terms['sl']
X = X.head(5000)
y = y.head(5000)
# Run classifier with cross-validation  
cv = StratifiedKFold(n_splits=6)
classifier = LogisticRegression(solver='lbfgs', max_iter=50)
tprs = []
aucs = []
mean_fpr = np.linspace(0, 1, 100)

fig, ax = plt.subplots()
for i, (train, test) in enumerate(cv.split(X, y)):
    #Fit the model
    classifier.fit(X.iloc[train], y.iloc[train])
    #Get the predictions and keep AUC values for later plot
    viz = plot_roc_curve(classifier, X.iloc[test], y.iloc[test],
                         name='ROC fold {}'.format(i),
                         alpha=0.3, lw=1, ax=ax)
    interp_tpr = interp(mean_fpr, viz.fpr, viz.tpr)
    interp_tpr[0] = 0.0
    tprs.append(interp_tpr)
    aucs.append(viz.roc_auc)

# Plot ROC curves        
ax.plot([0, 1], [0, 1], linestyle='--', lw=2, color='blue',
        label='Chance', alpha=.8)

mean_tpr = np.mean(tprs, axis=0)
mean_tpr[-1] = 1.0
mean_auc = auc(mean_fpr, mean_tpr)
std_auc = np.std(aucs)
ax.plot(mean_fpr, mean_tpr, color='green',
        label=r'Mean ROC (AUC = %0.2f $\pm$ %0.2f)' % (mean_auc, std_auc),
        lw=2, alpha=.8)

std_tpr = np.std(tprs, axis=0)
tprs_upper = np.minimum(mean_tpr + std_tpr, 1)
tprs_lower = np.maximum(mean_tpr - std_tpr, 0)
ax.fill_between(mean_fpr, tprs_lower, tprs_upper, color='grey', alpha=.2,
                label=r'$\pm$ 1 std. dev.')
#Now the standard model
standard_auc = roc_auc_score(y, standard_model['prob'])
#print('Standard: ROC AUC=%.3f' % (standard_auc))
base_fpr, base_tpr, _ = roc_curve(y, standard_model['prob'])
ax.plot(base_fpr, base_tpr, label=r'Standard ROC (AUC=%.3f)' % (standard_auc), color='red')
#Design of plot
ax.set(xlim=[-0.05, 1.05], ylim=[-0.05, 1.05],
       title="Logistic ROC compared to standard model")
ax.legend(loc="lower right")
plt.show()    